<a href="https://colab.research.google.com/github/ekrombouts/GenCareAI/blob/work_in_progress/scripts/work_in_progress/422_sampc_P_inference.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Cell 1: Install necessary libraries
!pip install -q transformers datasets

In [ ]:
# Cell 2: Import required libraries and mount Google Drive
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
from datasets import load_dataset

In [ ]:
# Cell 3: Load pre-trained fine-tuned model and tokenizer from Google Drive
model_finetuned = "ekrombouts/gcai_sampc_p_fietje"  # Path to your fine-tuned model
model = AutoModelForCausalLM.from_pretrained(model_finetuned, torch_dtype=torch.bfloat16, device_map='auto')
tokenizer = AutoTokenizer.from_pretrained(model_finetuned)
tokenizer.pad_token = tokenizer.eos_token  # Set pad token


In [ ]:
# Cell 4: Load dataset and retrieve sample 6
path_hf_sampc = "ekrombouts/Galaxy_SAMPC"
dataset = load_dataset(path_hf_sampc)
val_dataset = dataset['validation']
sample = val_dataset[6]

In [ ]:
# Cell 5: Prepare the prompt with truncated notes from sample 6
notes = sample['notes']  # Or use 'truncated_notes' if you already truncated it
prompt = f'''Lees de volgende rapportages en beschrijf de cognitie en gedragsproblemen van de cliënt.

Rapportages:
{notes}

Geef de output als lijst van strings, voorbeeld: ["aap", "noot", "mies"]

Beschrijf cognitie/gedragsproblemen:
'''


In [ ]:

# Cell 6: Tokenize the prompt
input_ids = tokenizer(prompt, return_tensors="pt").input_ids.to(model.device)
attention_mask = tokenizer(prompt, return_tensors="pt", padding=True).attention_mask.to(model.device)


In [ ]:

# Cell 7: Enable cache and set model to evaluation mode
model.config.use_cache = True
model.eval()


In [ ]:

# Cell 8: Generate output from the model
output = model.generate(
    input_ids,
    attention_mask=attention_mask,
    max_new_tokens=150,
    do_sample=True,
    top_p=0.95,
    top_k=50,
    temperature=0.7,
    num_return_sequences=1,
    eos_token_id=tokenizer.eos_token_id,
    pad_token_id=tokenizer.eos_token_id
)


In [ ]:

# Cell 9: Decode the generated text
generated_text = tokenizer.decode(output[0], skip_special_tokens=True)
generated_response = generated_text[len(prompt):].strip()


In [ ]:

# Cell 10: Display the generated response and actual response
ref_response = sample['psychisch']  # Reference response from dataset
print("GENERATED RESPONSE:")
print(generated_response)
print("\nREFERENCE RESPONSE:")
print(ref_response)

In [ ]:
print(notes)